In [3]:
from flask import Flask, request, jsonify
import requests
import os
import csv
from dotenv import load_dotenv
from PIL import Image,ImageDraw
from io import BytesIO
from roboflow import Roboflow
from models import Prediction  # Import the Prediction class
from utils import draw_bounding_boxes, calculate_total_price, load_prices, preprocess  # Import functions from utils.py

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="JQJQN8AUYAWhVEHwcIOv")
project = rf.workspace().project("dataset-waste-ddyct")
model = project.version(1).model

img_byte_arr, original_width, original_height = preprocess("./files/test1.jpg")

predictions_data = model.predict("./files/test1.jpg", confidence=40, overlap=30).json()
print(predictions_data)
predictions_list = []
for pred in predictions_data['predictions']:
            prediction = Prediction(
                x=pred['x'],
                y=pred['y'],
                width=pred['width'],
                height=pred['height'],
                confidence=pred['confidence'],
                class_name=pred['class'],
                class_id=pred['class_id'],
                detection_id=pred['detection_id'],
                image_path=pred['image_path']
            )
            predictions_list.append(prediction)
    
prices = load_prices()

total_price = calculate_total_price(predictions_list, prices)

print(total_price)

    # Prepare the image with bounding boxes
image = Image.open("./files/test1.jpg")
image_with_bboxes = draw_bounding_boxes(image, predictions_list, original_width, original_height)
image_with_bboxes.save('imagebbx.jpg', format='JPEG')

    # Save the image with bounding boxes to a byte buffer
img_byte_arr_with_bboxes = BytesIO()
image_with_bboxes.save(img_byte_arr_with_bboxes, format='JPEG')
img_byte_arr_with_bboxes.seek(0)
# infer on a local image
# predicitions = model.predict("464836539_936312811736506_4336630575233373391_n.jpg", confidence=40, overlap=30).json()
# print(predicitions)

# visualize your prediction
# model.predict("your_image.jpg", confidence=40, overlap=30).save("prediction.jpg")

# infer on an image hosted elsewhere
# print(model.predict("URL_OF_YOUR_IMAGE", hosted=True, confidence=40, overlap=30).json())

loading Roboflow workspace...
loading Roboflow project...
{'predictions': [{'x': 353, 'y': 507, 'width': 594, 'height': 702, 'confidence': 0.47958603501319885, 'class': 'Lawnmower', 'class_id': 7, 'detection_id': '3eced98a-c0fc-41a5-aff4-ebbecbb43eff', 'image_path': './files/test1.jpg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '728', 'height': '960'}}
15.0


0